In [ ]:
!apt-get update && sudo apt-get install -y build-essential git libgfortran3
!wget https://developer.nvidia.com/compute/cuda/10.0/Prod/local_installers/cuda-repo-ubuntu1804-10-0-local-10.0.130-410.48_1.0-1_amd64
!sudo dpkg -i cuda-repo-ubuntu1804-10-0-local-10.0.130-410.48_1.0-1_amd64.deb
!sudo apt-key add /var/cuda-repo-<version>/7fa2af80.pub
!sudo apt-get update
!sudo apt-get install cuda
!pip install mxnet-cu100

In [ ]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

In [ ]:
!mkdir d2lzh
!cp __init__.py ./d2lzh
!cp utils.py ./d2lzh
!ls ./d2lzh

In [1]:
import d2lzh as d2l
from mxnet import autograd,gluon,init,nd
from mxnet.gluon import nn

In [2]:
net=nn.Sequential()
net.add(nn.Conv2D(6,kernel_size=5),
       nn.BatchNorm(),
       nn.Activation('sigmoid'),
       nn.MaxPool2D(pool_size=2,strides=2),
       nn.Conv2D(16,kernel_size=5),
       nn.BatchNorm(),
       nn.Activation('sigmoid'),
       nn.MaxPool2D(pool_size=2,strides=2),
       nn.Flatten(),
       nn.Dense(120),
       nn.BatchNorm(),
       nn.Activation('sigmoid'),
       nn.Dense(84),
       nn.BatchNorm(),
       nn.Activation('sigmoid'),
       nn.Dense(10))

In [ ]:
d2l.try_gpu()

In [4]:
X=nd.random.uniform(shape=(1,1,224,224))
#net.initialize()
for layer in net:
    X=layer(X)
    print(layer.name,'shape is',X.shape)

conv0 shape is (1, 6, 220, 220)
batchnorm0 shape is (1, 6, 220, 220)
sigmoid0 shape is (1, 6, 220, 220)
pool0 shape is (1, 6, 110, 110)
conv1 shape is (1, 16, 106, 106)
batchnorm1 shape is (1, 16, 106, 106)
sigmoid1 shape is (1, 16, 106, 106)
pool1 shape is (1, 16, 53, 53)
flatten0 shape is (1, 44944)


MXNetError: Shape inconsistent, Provided = [120,256], inferred shape=(120,44944)

In [5]:
lr, num_epochs, batch_size, ctx = 1.0, 5, 256, d2l.try_gpu()
net.initialize(force_reinit=True,ctx=ctx, init=init.Xavier())
trainer = gluon.Trainer(net.collect_params(), 'sgd', {'learning_rate': lr})
train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size)
d2l.train_ch5(net, train_iter, test_iter, batch_size, trainer, ctx,
              num_epochs)

training on cpu(0)
epoch 1, loss 0.6443, train acc 0.770, test acc 0.827, time 17.1 sec
epoch 2, loss 0.3998, train acc 0.854, test acc 0.861, time 16.4 sec
epoch 3, loss 0.3464, train acc 0.874, test acc 0.849, time 17.3 sec
epoch 4, loss 0.3230, train acc 0.882, test acc 0.884, time 18.4 sec
epoch 5, loss 0.3039, train acc 0.889, test acc 0.826, time 18.1 sec
